In [1]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import tensorflow as tf
from tensorflow import keras

valence = pd.read_csv('deam-dataset/valence.csv').T
arousal = pd.read_csv('deam-dataset/arousal.csv').T

In [2]:
# retrieve feature data in dataframe format
pd_data = {}
i = []

for filename in os.listdir('deam-dataset/my_features/gemaps_data'):
    fn = os.path.join('deam-dataset/my_features/gemaps_data', filename)
    if '.csv' not in fn: continue
    f = open(fn, 'r')
    data = pd.read_csv(f, delimiter=',')
    
    # remove every alternate row (since each row represents a 0.01s time step, but the data for each row is for a 0.02s window)
    data = data.iloc[::2]
    if len(data.index) < 2251:
        i.append(int(filename[len('gemaps_'):].strip('.csv')))
        pad_size = 2251 - len(data.index)
        padding = data.iloc[[-1]*pad_size]
        data = data.append(padding)
    pd_data[int(filename[len('gemaps_'):].strip('.csv'))] = data.reset_index(drop=True)


In [3]:
len(pd_data[51].index)

2251

In [4]:
sample = pd_data[7]
sample

,file,start,end,Loudness_sma3,alphaRatio_sma3,hammarbergIndex_sma3,slope0-500_sma3,slope500-1500_sma3,spectralFlux_sma3,mfcc1_sma3,...,logRelF0-H1-A3_sma3nz,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2bandwidth_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3bandwidth_sma3nz,F3amplitudeLogRelF0_sma3nz
0,deam-dataset/DEAM_audio/MEMD_audio/7.mp3,0 days 00:00:00,0 days 00:00:00.020000,0.001034,0.000000,0.000000,0.132697,0.022186,0.000000,0.000000,...,0.000000,0.00000,0.0000,-201.000000,0.0000,0.00000,-201.000000,0.0000,0.00000,-201.000000
1,deam-dataset/DEAM_audio/MEMD_audio/7.mp3,0 days 00:00:00.020000,0 days 00:00:00.040000,0.001034,0.000000,0.000000,0.132697,0.022186,0.000000,0.000000,...,0.000000,0.00000,0.0000,-201.000000,0.0000,0.00000,-201.000000,0.0000,0.00000,-201.000000
2,deam-dataset/DEAM_audio/MEMD_audio/7.mp3,0 days 00:00:00.040000,0 days 00:00:00.060000,0.003787,-2.196436,6.190475,0.093206,0.017232,0.000242,5.962744,...,-85.338360,0.00000,0.0000,0.000000,0.0000,0.00000,0.000000,0.0000,0.00000,0.000000
3,deam-dataset/DEAM_audio/MEMD_audio/7.mp3,0 days 00:00:00.060000,0 days 00:00:00.080000,0.665254,-13.066643,25.597887,-0.008913,-0.006540,1.205779,25.589102,...,35.197464,911.48730,2094.2210,-29.225130,1604.0610,1075.07020,-37.828785,2801.8984,815.16130,-44.071686
4,deam-dataset/DEAM_audio/MEMD_audio/7.mp3,0 days 00:00:00.080000,0 days 00:00:00.100000,1.799997,-25.419365,35.791553,-0.083876,-0.014750,7.653868,28.354630,...,31.773256,458.40927,1762.6460,-32.398544,1490.3267,1221.00930,-44.369263,2394.9043,1459.25490,-48.684345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2246,deam-dataset/DEAM_audio/MEMD_audio/7.mp3,0 days 00:00:44.920000,0 days 00:00:44.940000,1.570490,-28.233002,38.489670,-0.066711,-0.011819,1.412865,22.473800,...,20.539352,795.37310,1205.8502,-38.293236,1852.8710,867.48810,-44.853960,2899.9521,560.33374,-47.146458
2247,deam-dataset/DEAM_audio/MEMD_audio/7.mp3,0 days 00:00:44.940000,0 days 00:00:44.960000,1.390248,-30.615723,40.819134,-0.075193,-0.012094,1.266584,26.096647,...,20.181839,744.52026,1248.7888,-38.610573,1861.1865,896.88995,-46.841442,2890.9314,891.23990,-47.012800
2248,deam-dataset/DEAM_audio/MEMD_audio/7.mp3,0 days 00:00:44.960000,0 days 00:00:44.980000,1.447859,-29.118400,42.248363,-0.084180,-0.009852,0.954308,22.171648,...,20.122885,1107.11710,1011.8564,-40.337370,2282.2083,792.99725,-47.044850,3223.7783,774.61914,-44.015118
2249,deam-dataset/DEAM_audio/MEMD_audio/7.mp3,0 days 00:00:44.980000,0 days 00:00:45,1.512686,-28.785421,41.633760,-0.090293,-0.006224,1.086609,18.152914,...,22.033173,1234.23650,1052.4918,-40.899265,2321.0500,814.64950,-45.276120,3218.3792,704.99730,-46.691273


In [5]:
sample.columns

Index(['file', 'start', 'end', 'Loudness_sma3', 'alphaRatio_sma3',
       'hammarbergIndex_sma3', 'slope0-500_sma3', 'slope500-1500_sma3',
       'spectralFlux_sma3', 'mfcc1_sma3', 'mfcc2_sma3', 'mfcc3_sma3',
       'mfcc4_sma3', 'F0semitoneFrom27.5Hz_sma3nz', 'jitterLocal_sma3nz',
       'shimmerLocaldB_sma3nz', 'HNRdBACF_sma3nz', 'logRelF0-H1-H2_sma3nz',
       'logRelF0-H1-A3_sma3nz', 'F1frequency_sma3nz', 'F1bandwidth_sma3nz',
       'F1amplitudeLogRelF0_sma3nz', 'F2frequency_sma3nz',
       'F2bandwidth_sma3nz', 'F2amplitudeLogRelF0_sma3nz',
       'F3frequency_sma3nz', 'F3bandwidth_sma3nz',
       'F3amplitudeLogRelF0_sma3nz'],
      dtype='object')

In [6]:
# with open('mel_spectogram_data.npy', 'rb') as f:
#     mel_spec_data = np.load(f)
# print(f'Mel spec data shape: {mel_spec_data.shape}')

with open('mel_spectogram_data.npy', 'rb') as f:
    mfcc_data = np.load(f)
print(f'MFCC data shape: {mfcc_data.shape}')

MFCC data shape: (1802, 1920, 128)


In [7]:
import math

# input: time-step (in seconds), DataFrame to convert
# outpu: converted dataframe
def recalibrate_intervals(input_df, time_step):
    if 45 % time_step != 0: 
        raise ValueError('45 not divisible by ' + str(time_step))
    window_size = math.ceil(time_step / 0.02)
    lst = []
    rolling = input_df.rolling(window_size).mean()
    for i in range(0, 2250, window_size):
        end_i = i + window_size - 1
        row = rolling.iloc[end_i]
        row['start'] = input_df.iloc[i, input_df.columns.get_loc('start')]
        row['end'] = input_df.iloc[end_i, input_df.columns.get_loc('end')]
        lst.append(row)
    cols = input_df.columns
    df = pd.DataFrame(lst)
    df = df.loc[:, cols].reset_index(drop=True).drop('file', axis=1)
    return df



In [8]:
re_sample = recalibrate_intervals(sample, 0.5).drop(['start', 'end'], axis=1)
re_sample.to_numpy().shape

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


(90, 25)

In [18]:
re_sample

,Loudness_sma3,alphaRatio_sma3,hammarbergIndex_sma3,slope0-500_sma3,slope500-1500_sma3,spectralFlux_sma3,mfcc1_sma3,mfcc2_sma3,mfcc3_sma3,mfcc4_sma3,...,logRelF0-H1-A3_sma3nz,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2bandwidth_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3bandwidth_sma3nz,F3amplitudeLogRelF0_sma3nz
0,1.328668,-26.983522,36.633498,-0.078930,-0.012349,4.069342,29.442273,23.387838,22.317323,17.506208,...,27.014581,575.824452,1219.814464,-38.889451,1615.618056,1005.766415,-49.154057,2550.342320,1036.763839,-56.258943
1,1.607192,-28.084900,40.537119,-0.077069,-0.021459,3.571981,34.054681,25.666058,19.699260,15.306500,...,39.803997,498.416594,1437.816488,-18.039729,1538.176376,1255.873457,-33.454395,2642.599608,1112.131750,-41.834678
2,1.570784,-28.478644,39.001255,-0.076076,-0.019849,3.331796,32.407694,24.163669,21.522524,15.715705,...,38.537076,451.379223,1446.441228,-22.550353,1498.490408,1202.274602,-36.233821,2562.411112,1016.438362,-44.184699
3,1.997997,-22.235085,32.140286,-0.050504,-0.011852,3.721461,27.321374,20.575901,17.584617,14.783716,...,31.389754,473.129234,1305.444684,-10.340211,1497.362356,1053.384584,-26.756457,2521.835556,800.407315,-32.594961
4,1.588692,-27.603085,39.283250,-0.124693,-0.007163,4.699239,22.491248,25.191997,26.278216,24.121592,...,36.186678,911.607063,1270.797674,-25.268921,1946.043560,1064.373527,-36.025575,3035.813276,838.476097,-40.986997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,1.548559,-27.013224,37.323025,-0.112213,-0.008890,3.285683,25.411907,23.410454,26.133198,19.987429,...,36.668308,647.902542,1380.701068,-24.344401,1749.849060,1042.038153,-35.095809,2779.416956,845.193862,-38.592476
86,1.773968,-23.195840,33.474635,-0.090380,-0.005882,2.726020,21.826910,19.432415,22.822677,16.413431,...,29.553507,1030.315819,1092.530612,-28.602576,2190.704008,940.764642,-35.561867,3119.845636,876.574453,-36.658004
87,1.928227,-22.930624,31.043239,-0.081138,-0.014477,4.126724,23.477347,20.625436,19.787410,15.126010,...,32.806741,571.046682,1405.397592,-25.726966,1631.617576,962.607138,-38.460763,2734.099932,726.918797,-42.190540
88,1.790687,-23.245632,32.992397,-0.089007,-0.007754,3.119768,23.178699,19.058086,20.697829,19.235227,...,33.231318,748.330192,1264.321591,-23.936010,1838.531132,911.671799,-33.319108,2929.615504,798.260264,-37.033406


In [9]:
re_sample.columns

Index(['Loudness_sma3', 'alphaRatio_sma3', 'hammarbergIndex_sma3',
       'slope0-500_sma3', 'slope500-1500_sma3', 'spectralFlux_sma3',
       'mfcc1_sma3', 'mfcc2_sma3', 'mfcc3_sma3', 'mfcc4_sma3',
       'F0semitoneFrom27.5Hz_sma3nz', 'jitterLocal_sma3nz',
       'shimmerLocaldB_sma3nz', 'HNRdBACF_sma3nz', 'logRelF0-H1-H2_sma3nz',
       'logRelF0-H1-A3_sma3nz', 'F1frequency_sma3nz', 'F1bandwidth_sma3nz',
       'F1amplitudeLogRelF0_sma3nz', 'F2frequency_sma3nz',
       'F2bandwidth_sma3nz', 'F2amplitudeLogRelF0_sma3nz',
       'F3frequency_sma3nz', 'F3bandwidth_sma3nz',
       'F3amplitudeLogRelF0_sma3nz'],
      dtype='object')

In [10]:
keys = list(pd_data.keys())
keys.sort() # arrange in ascending order (same as mfcc data)

cols = list(pd_data[2].columns)
cols_to_drop = list(filter(lambda col: 'mfcc' in col, cols))
cols_to_drop.extend(['start', 'end'])
cols_to_drop = np.array(cols_to_drop)
n_features = len(pd_data[2].columns) - len(cols_to_drop) - 1 # 1 representing 'file' column, which is removed in recalibrate_intervals()

def process_gemaps(time_step=0.5):
    final = np.empty((len(keys), int(45 / time_step), n_features))
    for i, key in enumerate(keys):
        re_data = recalibrate_intervals(pd_data[key], time_step)
        final[i] = re_data.drop(cols_to_drop, axis=1).to_numpy(dtype=np.float32)
    return final

In [11]:
# fetch labels
label_path = 'deam-dataset/DEAM_Annotations/annotations/annotations averaged per song/song_level/';
labels_1_2000 = pd.read_csv(os.path.join(label_path, 'static_annotations_averaged_songs_1_2000.csv'))
labels_2000_2058 = pd.read_csv(os.path.join(label_path,'static_annotations_averaged_songs_2000_2058.csv'))
labels = pd.concat([labels_1_2000, labels_2000_2058], ignore_index=True, sort=False)
labels = labels[labels_1_2000.columns]
labels.columns = labels.columns.str.replace(' ', '')

valence = labels['valence_mean'].to_numpy()
arousal = labels['arousal_mean'].to_numpy()
y = np.vstack([valence, arousal]).T
y.shape

(1802, 2)

### 0.5 second intervals

In [12]:
gemaps_data = process_gemaps(time_step=0.5)

In [13]:
gemaps_data.shape

(1802, 90, 21)

In [14]:
from sklearn.model_selection import train_test_split

idx = np.arange(gemaps_data.shape[0])
X_trn_idx, X_test_idx, y_trn, y_test = train_test_split(idx, y, test_size=0.2, random_state=23)
X_train_idx, X_valid_idx, y_train, y_valid = train_test_split(X_trn_idx, y_trn, test_size=0.33, random_state=23)

X_gemaps_train = gemaps_data[X_train_idx]
X_gemaps_valid = gemaps_data[X_valid_idx]
X_gemaps_test = gemaps_data[X_test_idx]

In [15]:
gemaps_data.shape[1], gemaps_data.shape[2]

(90, 21)

In [16]:
from functools import partial
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, 
                                     LSTM, RepeatVector,
                                     Dense, 
                                     Flatten, Dropout, 
                                     LayerNormalization, BatchNormalization)

input_shape = gemaps_data.shape[1], gemaps_data.shape[2]
lstm_model = keras.Sequential([
    LayerNormalization(),
    LSTM(64, input_shape=input_shape, return_sequences=True),
    LSTM(128),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(2)
])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

optimizer = keras.optimizers.Adam(lr=0.00001)
lstm_model.compile(loss='mae', optimizer=optimizer, metrics=['mean_squared_error'])
history = lstm_model.fit(X_gemaps_train, y_train, 
                    epochs=100,
                    validation_data=(X_gemaps_valid, y_valid), 
                    batch_size=16, 
                    callbacks=[early_stopping])  

Epoch 1/100
61/61 [==============================] - 2s 27ms/step - loss: 4.7125 - mean_squared_error: 23.7568 - val_loss: 4.5397 - val_mean_squared_error: 22.0985
Epoch 2/100
61/61 [==============================] - 1s 15ms/step - loss: 4.4713 - mean_squared_error: 21.5619 - val_loss: 4.2645 - val_mean_squared_error: 19.6944
Epoch 3/100
61/61 [==============================] - 1s 16ms/step - loss: 4.1323 - mean_squared_error: 18.6952 - val_loss: 3.8335 - val_mean_squared_error: 16.2328
Epoch 4/100
61/61 [==============================] - 1s 15ms/step - loss: 3.5681 - mean_squared_error: 14.4588 - val_loss: 3.1167 - val_mean_squared_error: 11.3128
Epoch 5/100
61/61 [==============================] - 1s 15ms/step - loss: 2.7602 - mean_squared_error: 9.4479 - val_loss: 2.2291 - val_mean_squared_error: 6.4516
Epoch 6/100
61/61 [==============================] - 1s 15ms/step - loss: 2.0236 - mean_squared_error: 5.6346 - val_loss: 1.6376 - val_mean_squared_error: 3.7337
Epoch 7/100
61/61 [=

In [17]:
from sklearn.metrics import accuracy_score, classification_report

def categorize_val_arousal(data):
    cat_data = []
    # A: active, P: passive
    # P: positive, N: negative
    for point in data:
        valence, arousal = point[0], point[1]
        if valence >= 5 and arousal >= 5:
            cat_data.append('AP')
        elif valence < 5 and arousal >= 5:
            cat_data.append('AN')
        elif valence >= 5 and arousal < 5:
            cat_data.append('PP')
        elif valence < 5 and arousal < 5:
            cat_data.append('PN')
    return np.array(cat_data, dtype='str')

score = lstm_model.evaluate(X_gemaps_test, y_test)
print(f'Test reg RMSE: {list(map(math.sqrt, score))}')
y_pred = lstm_model.predict(X_gemaps_test)

cat_y_pred = categorize_val_arousal(y_pred)
cat_y_test = categorize_val_arousal(y_test)

accuracy = accuracy_score(cat_y_test, cat_y_pred)
print(f'Test Accuracy: {accuracy}')

12/12 [==============================] - 0s 5ms/step - loss: 1.0312 - mean_squared_error: 1.4943
Test reg RMSE: [1.0154684679741504, 1.222435489504856]
Test Accuracy: 0.3490304709141274
